<a href="https://colab.research.google.com/github/Polaris0116/awesome_lists/blob/main/003_final_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 注意事项
## 最后的结果输出在本文件的最后一部分，可以看到相应的文本输出

## 1. Load Testing Data

In [ ]:
# Basic Libaries
import os
import numpy as np
import pandas as pd

# Data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Neural Networks
from tensorflow import keras
import tensorflow.keras.layers as layers

# Early Stopping
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Check the files
train_dir = "/Users/steven/Downloads/upload/dataset"
test_dir = "/Users/steven/Downloads/upload/test"

train_files = os.listdir(train_dir)
test_files = sorted(os.listdir(test_dir))

train_paths = [os.path.join(train_dir, filename) for filename in train_files]
test_paths = [os.path.join(test_dir, filename) for filename in test_files]

In [ ]:
# Create a merged traninig dataset
merged_train = pd.DataFrame()
for each_file in train_paths:
    data = pd.read_csv(each_file, header=None)
    merged_train = pd.concat([merged_train, data], axis=1, ignore_index=True)

# Transpose the whole dataset
transposed_train = merged_train.transpose()
# Print the traning shape
print(f"The traning shape is: {transposed_train.shape}")

# Create a merged testing dataset
merged_test = pd.DataFrame()
for each_file in test_paths:
    data = pd.read_csv(each_file, header=None)
    merged_test = pd.concat([merged_test, data], axis=1, ignore_index=True)

# Transpose the whole dataset
transposed_test = merged_test.transpose()
# Print the testing shape
print(f"The testing shape is: {transposed_test.shape}")

The traning shape is: (90, 8192)
The testing shape is: (110, 8192)


In [ ]:
# Take a glimpse of the testing dataset
transposed_test.head(5)

,0,1,2,3,4,5,6,7,8,9,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
0,0.590819,-0.210481,-0.406121,-0.029921,0.473089,-0.745891,0.202489,-0.140941,-0.239221,-0.097631,...,-0.228001,-0.167001,0.175649,0.171749,-0.068421,0.097939,-0.269651,-0.270941,0.384029,-0.204821
1,1.359604,-1.049046,2.258914,0.451314,0.421424,-2.173106,3.413474,-0.958156,-1.510036,-1.558396,...,0.375234,-1.679616,1.407184,0.260384,0.675964,0.891124,-0.836766,-0.103006,0.362864,-1.851636
2,0.103542,-0.008088,-0.376480,0.226518,0.209404,-0.051018,0.399724,-0.041394,-0.712712,0.122296,...,0.202132,-0.065780,0.190162,0.121354,-0.010636,0.088258,0.097564,-0.164844,0.213952,-0.128576
3,0.153252,0.513152,-0.162388,-0.946848,-0.597928,0.264272,-0.302688,0.757152,0.061752,-0.122128,...,-0.496668,0.155692,0.737632,-0.381988,0.022712,0.420432,-0.190448,-0.700408,-0.011448,-0.400288
4,0.068773,0.500043,-0.175789,0.234449,0.135595,0.078913,0.060125,-0.495673,0.049619,0.069227,...,0.666783,-0.270129,-0.204947,-0.204825,0.218555,0.253569,-0.015285,0.222947,0.300573,-0.309745


In [ ]:
# Create y-label for training set
train_label = []
train_label
for each_file in train_files:
    label = int(each_file[-5])
    train_label.append(label)

# Merge into the training dataset
transposed_train["target"] = train_label

# Hyper-prameters
NUM_CLASSES = 2

# Hyper-parameters for tranining process (Not the model params)
EPOCH = 25
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.1

# Define training features and labels
X_train = transposed_train.drop(["target"], axis=1).values
y_train = transposed_train["target"].values
# Define testing features
X_test = transposed_test.values

# Normalization of the data
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert y-train to one-hot
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)

# Print training shape
print(X_train_normalized.shape)
print(y_train.shape)

(90, 8192)
(90, 2)


## 2. Load Best Model File

In [ ]:
# Load the best model without compiling i.e. Model_A
best_model = keras.models.load_model(f"***best_model_Final***.keras", compile=False)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 96)                786528    
                                                                 
 dense_1 (Dense)             (None, 84)                8148      
                                                                 
 dense_2 (Dense)             (None, 42)                3570      
                                                                 
 dense_3 (Dense)             (None, 2)                 86        
                                                                 
Total params: 798,332
Trainable params: 798,332
Non-trainable params: 0
_________________________________________________________________


2023-12-10 02:51:52.597959: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 3. Prediction Result

In [ ]:
# Prediction on the testing set.
prediction = best_model.predict(X_test_normalized) # To speed up, uesing batch_size=BATCH_SIZE
prediction.shape

(110, 2)

In [ ]:
# Prediction Result
predict_value = np.argmax(prediction, axis=1)
predict_value

array([1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1])

array([1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1])

In [ ]:
result_list = list(predict_value)
result_str = ""
for each_value in result_list:
    result_str = result_str + str(each_value)

# Convert result to string
print(len(result_str))

110


## 4. Result Output

In [ ]:
# Open a new file
file_path = "***result***.txt"
file = open(file_path, 'w')

# Write Data
data = result_str
for char in data:
    file.write(char + '\n')

# Close file
file.close()

# Check if the result txt file exists
if os.path.isfile(f"***result***.txt") == True:
    print("The prediction result has been saved successfully.")
else:
    print("Error Occured")

The prediction result has been saved successfully.


In [ ]:
# Read file: result.txt
with open(file_path, 'r') as file:
    content = file.read()
    print(content)

1
1
0
1
0
0
0
0
1
1
1
1
1
0
1
1
0
1
0
1
1
1
0
1
1
1
0
0
1
1
1
0
1
0
0
1
1
0
1
1
1
0
1
0
1
1
0
0
0
1
0
1
0
1
0
1
1
1
0
0
1
0
0
1
1
1
0
1
1
1
1
0
0
1
0
0
0
1
0
1
0
1
1
0
1
1
1
1
0
1
1
0
0
1
0
0
1
1
0
0
0
0
0
1
1
0
0
0
1
1

